In [1]:
# Utility script

# geometric_transf

In [2]:
import numpy as np
import cv2
from math import floor, ceil
import random

In [3]:
import numpy as np

def newPix_by_RadiationPattern(im, pos, Rz_ri):
    # Evaluate new pixel value by radiation pattern model.
    #
    #   input:
    # im - high resolution image;
    # pos - new pixel coordinates respactive to im;
    # Rz_ri - averaging radius (for circular zone).
    #
    #   output:
    # pixVal - new pixel value.
  
    
    rad_pattern = 'sinc';  # 'gauss', 'sinc'      # XXX - Attension
    
    n = np.size(im,0)
    m = np.size(im,1)
   
    
    # Check if the zone (circle with radius Rz_ri) goes beyond im.
    tol = 1e-9;
    if (pos[0]+Rz_ri > n-1+tol)  or  (pos[1]+Rz_ri > m-1+tol)  or  \
       (pos[0]-Rz_ri < 0-tol)  or  (pos[1]-Rz_ri < 0-tol):
        raise ValueError("Zone goes beyond im")

    # Y - grid for columns.
    [Y,X] = np.meshgrid(np.arange(m), np.arange(n))          
    
    if rad_pattern == 'gauss':
        gam = 1.2/Rz_ri  
        # Gaussian RBF.
        f = lambda x : np.exp(-(gam*x)**2) 
    elif rad_pattern == 'sinc':
        f = lambda x : ( np.sin(np.pi*x/Rz_ri) / (np.pi*x/Rz_ri) )**2
    else:
        raise ValueError("Wrong rad_pattern")
    
    R = np.sqrt( (X-pos[0])**2 + (Y-pos[1])**2 )
    
    with np.errstate(divide='ignore', invalid='ignore'):
        W = f(R)
    
    if rad_pattern == 'sinc':
        W[np.isnan(W)] = 1
    
    # "0" for values outside ZONE.
    W[R >= Rz_ri] = 0
    
    # Normalize weights.                                        
    W = W / np.sum(W)                   # XXX - Attension.

    
    # ------- DEBUG ----------
    # from matplotlib import pyplot as plt
    # from matplotlib import cm
    # from mpl_toolkits.mplot3d import Axes3D
    
    # fig = plt.figure()
    # ax = fig.gca(projection='3d')
    
    # # cmap_ = cm.coolwarm
    # cmap_ = cm.copper
    # surf = ax.plot_surface(X/5, Y/5, W,
    #                       antialiased=False,
    #                       cmap=cmap_)

    # ax.set_xlabel('X', fontsize =20, labelpad=18)
    # ax.set_ylabel('Y', fontsize =20, labelpad=18)
    # ax.set_zlabel('Radiation pattern', fontsize =20, labelpad=18)
    # ax.tick_params(labelsize = 18)
    # # fig.colorbar(surf, shrink=0.5, aspect=5)
    # plt.show()
    # ---------------------
        
    wIm = im*W
    pixVal = np.sum(wIm)
    return pixVal

In [4]:
def make_radPattern_im(ri_HR,ri_s,ci_s,
                       Rz_ri,
                       shift_x,shift_y,
                       scale,rot):
    '''
    Forms a reference (RI) and current (CI) images based on radiation pattern
        model. It simulates the shift, rotation and rescaling.
        
    Input:
        ri_HR - high resolution reference image;
        ri_s - (after averaging, pixel formation) reference image size;     
        ci_s - (after averaging, pixel formation) current image size;  
        Rz_ri - averaging radius (for circular zone);
        shift_x, shift_y - subpixel shift along X and Y axes;
        scale, rot - scaling and rotation coeficients
                    (clockwise rotation for grid coordinates,
                     visually it seems that we counter-clockwise rotate the picture)?
                rot in degrees.

    Output:
        ri - reference image (after averaging, pixel formation);
        ci_clean - current image (after averaging, pixel formation), "clean" means without noise;
        center_x, center_y - coordinates of ci_clean center relative to ri grid.
    
    Comments:
        ri_HR - should be big enough.
        We think that RI and CI images are square. ri_HR may be rectangular.
    '''
    
    
    debug_mode = 'no'     # 'yes'  ,   'no'
    
    
    
    def new_pix_on_fragm(ri_HR,dx,dy,rZone):
        x_1 = floor(dx-rZone)
        x_end = ceil(dx+rZone)
        y_1 = floor(dy-rZone)
        y_end = ceil(dy+rZone)
        
        pos = [dx-x_1,
               dy-y_1]
        im = ri_HR[x_1 : x_end+1,
                   y_1 : y_end+1]
        new_pix = newPix_by_RadiationPattern(im,pos,rZone)
        
        return new_pix
    
    
    #%%
    ri = np.zeros( (ri_s,ri_s) )
    ci_clean = np.zeros( (ci_s,ci_s) )
    
    # --- DEBUG ---
    if debug_mode == 'yes':
        ri_x_grid = np.zeros( (ri_s,ri_s) )
        ri_y_grid = np.zeros( (ri_s,ri_s) )
        ci_clean_x_grid  = np.zeros( (ci_s,ci_s) )
        ci_clean_y_grid = np.zeros( (ci_s,ci_s) )
    # -------------
    
    
    step_ri = Rz_ri         #  >2*Rz_ri   means without overlap.   
    Rz_ci = scale*Rz_ri
    # step_ci = Rz_ci  
    
    # Find center position of CI in HR image grid.
    (n,m) = ri_HR.shape
    centerHR_x = n/2-0.5
    centerHR_y = m/2-0.5
    
    # RI initial random shift (within HR pixel)
    # (to get closer to the real situation).
    centerHR_x = centerHR_x + random.uniform(-0.5,0.5)                 
    centerHR_y = centerHR_y + random.uniform(-0.5,0.5) 
    
    
    #%%
    ini_x = centerHR_x - step_ri*(ri_s-1)/2
    ini_y = centerHR_y - step_ri*(ri_s-1)/2
    
    xShifts = ini_x + np.arange(ri_s)*step_ri
    yShifts = ini_y + np.arange(ri_s)*step_ri
    
    for ix in range(ri_s):
        for iy in range(ri_s):
            dx = xShifts[ix]
            dy = yShifts[iy]
            ri[ix,iy] = new_pix_on_fragm(ri_HR,dx,dy,Rz_ri)
            #
            # --- DEBUG ---
            if debug_mode == 'yes':
                ri_x_grid[ix,iy] = dx
                ri_y_grid[ix,iy] = dy
            # -------------
            
            
    #%% CI points positions.      
                
    # XXX - Alternative.
    #ini_x = centerHR_x - step_ci*(ci_s-1)/2                                    
    #ini_y = centerHR_y - step_ci*(ci_s-1)/2
    #
    #ci_xShifts = ini_x + np.arange(ci_s)*step_ci
    #ci_yShifts = ini_y + np.arange(ci_s)*step_ci
    #
    #tM = cv2.getRotationMatrix2D( (centerHR_x,centerHR_y), rot,1)
    # --------------
    
    
    ini_x = centerHR_x - step_ri*(ci_s-1)/2
    ini_y = centerHR_y - step_ri*(ci_s-1)/2
    
    ci_xShifts = ini_x + np.arange(ci_s)*step_ri
    ci_yShifts = ini_y + np.arange(ci_s)*step_ri
    
    # Transformation matrix for HR (high resolution) grid coordinates.
    tM = cv2.getRotationMatrix2D( (centerHR_x,centerHR_y), rot,scale)
    
    
   
    
    # --- DEBUG ---
    # # Test function for scale,rot extraction from transformation matrix.
    # from functions.registration.transf_2stage_estim import extract__scale_rot
    
    # (sc_,rot_) = extract__scale_rot(tM)
    # print('True = ',scale,rot)
    # print('Est = ',sc_,rot_)
    # ------------
    
    
    
    
    
    #%%
    for ix in range(ci_s):
        for iy in range(ci_s):
            # Evaluate CI pixel.
            dx = ci_xShifts[ix]
            dy = ci_yShifts[iy]
            src = np.array([ [dx],
                             [dy],
                             [1] ])
            dst = np.dot(tM, src)
            dx = dst[0] + shift_x*step_ri
            dy = dst[1] + shift_y*step_ri
            
            # dx and dy - arrays with one element.
            ci_clean[ix,iy] = new_pix_on_fragm(ri_HR, dx[0], dy[0], Rz_ci)
            #
            # --- DEBUG ---
            if debug_mode == 'yes':
                ci_clean_x_grid[ix,iy] = dx
                ci_clean_y_grid[ix,iy] = dy
            # -------------
    
    # --- DEBUG ---
    if debug_mode == 'yes':
        from matplotlib import pyplot as plt
        plt.figure(figsize = (5,5) )
        plt.plot(ri_y_grid, ri_x_grid, 'ro')
        plt.plot(ci_clean_y_grid, ci_clean_x_grid, 'g^')
        plt.ylim(n-1,0)
        plt.xlim(0,m-1)
        plt.grid()
        plt.xlabel('Y')
        plt.ylabel('X')
        plt.show()
    # -------------
    
    
    
    # Find center position of CI in ri image grid.
    center_x = ri_s/2-0.5 + shift_x
    center_y = ri_s/2-0.5 + shift_y
    
    return (ri,ci_clean,
            center_x, center_y)